Select Hospital Data - https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/anag-cw7u

CDC De-identified individual cases - 9gb File!: https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Public-Use-Data-with-Ge/n8mc-b4w4




# Download and Build Combined COVID Datasets #
This process takes roughly 40 minutes to complete.

### DataSets ###
**John's Hopkins COVID Cases:** https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv

**John's Hopkins COVID Deaths:** https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv 

**CDC U.S. Vaccine Data:** https://data.cdc.gov/api/views/unsk-b7fc/rows.csv?accessType=DOWNLOAD

**CDC County Vaccine Data:** https://data.cdc.gov/api/views/8xkx-amqh/rows.csv?accessType=DOWNLOAD

**State Hospitalization Data:** https://healthdata.gov/api/views/g62h-syeh/rows.csv?accessType=DOWNLOAD

In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import time

from importlib import reload #//*** Reload library reloads an external python File.

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import clear_output

from datetime import datetime
from pathlib import Path
import os


pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

#//*** Custom Functions outboarded here for Brevity
import process_covid


data_folder_name = "raw_data"
#//***  Raw and Processed Filenames
confirmed_data_filename = "z_us_confirmed.csv"
death_data_filename =  "z_us_death_cases.csv"
county_daily_df_filename = "z_county_daily_df.csv.zip"
aggregate_hospital_filename = "z_county_hospital_aggregate.csv.zip"
combined_daily_casevax_filename = "z_county_daily_casevax.csv.zip"
weekly_combined_filename = "z_county_weekly_df.csv.zip"

#//*** Weekly Combined Cases,Deaths, Vaccines, Hospital
weekly_combined_cdvh_df_filename = "z_county_weekly_combined_cdvh_df.csv.zip"


In [4]:

reload(process_covid)


#//******************************************************
#//*** Begin Full Download and Data set Processing
#//******************************************************

start_time = time.time()
#//***************************************************************************************************
#//*** Download Latest Data - Updates Daily for Confirmed And Deaths
#//***************************************************************************************************
pc = process_covid.download_data()

print(f"Data Downloaded: {int(time.time() - start_time)}s")

#//*** Merge County Level Cases and Deaths. Converts Column based Days to Row based Days.
process_covid.build_county_case_death(
    folder = data_folder_name,
    confirm = confirmed_data_filename,
    death = death_data_filename,
    export = county_daily_df_filename,
)

print(f"Total Elapsed Time: {int(time.time() - start_time)}s")

#//***************************************************************************************************
#//*** Build Weekly Aggregated hospital Values. Sum all hospital stats for each county by Week
#//*** Exports to File
#//***************************************************************************************************
#//*** 4 - 5 Minute Process!
#//***************************************************************************************************
#//*** Load Raw Hospital Data
raw_hosp_df = process_covid.load_data(filename="z_county_hospital.csv.zip")

#//*** Columns to Display
disp_cols = ['collection_week', 'fips_code','state', 'total_beds_7_day_avg','inpatient_beds_7_day_avg','inpatient_beds_used_7_day_avg', 'inpatient_beds_used_covid_7_day_avg', 'total_icu_beds_7_day_avg', 'icu_beds_used_7_day_avg','staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg']

#//*** Descriptive Classification Columns: Will take a single value
base_cols = ['collection_week', "fips_code",'state']

#//*** Columns to Process, ie sum 
process_cols = ['total_beds_7_day_avg','inpatient_beds_7_day_avg','inpatient_beds_used_7_day_avg', 'inpatient_beds_used_covid_7_day_avg', 'total_icu_beds_7_day_avg', 'icu_beds_used_7_day_avg','staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg']


#//***************************************************************************************************
#//*** Aggregate (sum) Raw Hospital Data
#//*** Combine all Hospitals in each county on a given day.
#//***************************************************************************************************
process_covid.aggregate_columns(raw_hosp_df,
    by = "fips_code", #//*** All Granular Values to sum. All Hospitals in a County
    date_col = 'collection_week', #//*** Date Column 
    method = 'sum', #//*** Add Columns
    base_cols = base_cols, #//** Descriptive Columns
    process_cols = process_cols, #//*** Columns to Sum
    disp_cols = disp_cols, #//*** Columns to Display
    folder="raw_data",
    export=aggregate_hospital_filename, #//*** Filename to export DataFrame Too
)
print(f"Total Elapsed Time: {int(time.time() - start_time)}s")


#//***************************************************************************************************
#//*** Merge Cases & Deaths with Vaccines
#//*** Daily Interval, by County
#//***************************************************************************************************

#//*** Load Vaccine Data. County Level, Daily Interval
county_vax_df = process_covid.load_data(action="county_vaccine")
display(county_vax_df)

#//*** Load Confirmed Cases and Deaths
county_daily_df = process_covid.load_data(file=county_daily_df_filename, min_date=county_vax_df['Date'].min())
county_daily_df = county_daily_df[county_daily_df["Population"] > 0]
display(county_daily_df)

#//*** Merge Daily Cases
process_covid.merge_df(county_vax_df,county_daily_df,by="Date",left_col="FIPS",right_col="FIPS",export=combined_daily_casevax_filename)

#//***************************************************************************************************
#//*** Convert Combined Cases & Deaths & Vaccines
#//*** To Weekly Interval from Daily
#//*** by County 
#//***************************************************************************************************

#//*** Load the Combined Daily Cases and Vaccinations by County
daily_casevax_df = process_covid.load_data(filename=combined_daily_casevax_filename)

    #//************************************************************
    #//*** Load aggregated (summed by county) Hospital Data
    #//************************************************************

#//*** Load Weekly Aggregated Hospital by County
week_hosp_df = process_covid.load_data(filename=aggregate_hospital_filename)


#//*** Convert county daily to weekly interval
#//*** 10 Minute Build
process_covid.create_weekly_data(daily_casevax_df,export=weekly_combined_filename,dates=week_hosp_df['collection_week'].unique() )

#//***************************************************************************************************
#//*** Combine Aggregated Hospital Data with Weekly Cases & Deaths & Vaccines
#//*** Weekly Interval
#//*** By County
#//***************************************************************************************************

#//*** Load Weekly Case Death Vax by County
casevax_weekly_df = process_covid.load_data(filename=weekly_combined_filename)



#//*** Convert collection_week to datetime
week_hosp_df["collection_week"] = pd.to_datetime(week_hosp_df["collection_week"])

#//*** Rename columns: collection_week --> Date
week_hosp_df.columns = ['Date' if item == 'collection_week' else item for item in week_hosp_df.columns]
#//*** Rename columns: fips_code --> FIPS
week_hosp_df.columns = ['FIPS' if item == 'fips_code' else item for item in week_hosp_df.columns]

#//*** Merge Hospital and Case Death Vax Dataframes
process_covid.merge_df(casevax_weekly_df, #//*** Left Df
                       week_hosp_df,      #//*** Right DF
                       by="Date",         #//*** Aggregate using By Column (should always be a date column since this is a timeseries)
                       left_col="FIPS",   #//*** Left Col to Merge
                       right_col="FIPS",  #//*** Right Col to Merge
                       remove_cols = ['Recip_County'], #//*** Remove these columns for tidiness
                       export=weekly_combined_cdvh_df_filename)


print(f"DONE! Total Elapsed Time: {int(time.time() - start_time)}s")


Elapsed Time: 1s
Exporting DataFrame to Disk.
DONE! Total Elapsed Time: 1260s


## Code Below is Designed to fail. Want to preserve the Notebook Output ##


In [71]:
del fail
fail

NameError: name 'fail' is not defined

In [13]:
if False:
    #//*** The original dataset is 9gb and comprises 69 million rows.
    #//*** The data has been split into separate dataframes by state, compressed and pickled.
    #//*** Process patient impact parses those files, and sum s
    reload(process_covid)


    pc = process_covid.process_covid()

    #//*** Build aggregated Dataframe from the original split dataset
    df = pc.process_patient_impact(

        #//*** Names of the Columns to Keep 
        base_cols = ["case_month","res_state","state_fips_code","res_county","county_fips_code"],

        #//*** Columns to Process (Sum)
        process_cols = ['hosp_yn','icu_yn','death_yn','underlying_conditions_yn'],

        #//*** Rename columns using a tuple
        rename_cols = [('hosp_yn','hosp'),('icu_yn','icu'),('death_yn','death'),('underlying_conditions_yn','uc')],
    )
    df.to_pickle(f"./raw_data/summed_outcomes.pkl.zip")

Processing: AK_covid_people.pkl.zip
Loaded:  236740  records Current Record Size: 0
Processing: AL_covid_people.pkl.zip
Loaded:  1282352  records Current Record Size: 121
Processing: AR_covid_people.pkl.zip
Loaded:  827425  records Current Record Size: 1258
Processing: AZ_covid_people.pkl.zip
Loaded:  2006278  records Current Record Size: 2051
Processing: CA_covid_people.pkl.zip
Loaded:  9075259  records Current Record Size: 2376
Processing: CO_covid_people.pkl.zip
Loaded:  1342246  records Current Record Size: 3610
Processing: CT_covid_people.pkl.zip
Loaded:  454782  records Current Record Size: 4247
Processing: DC_covid_people.pkl.zip
Loaded:  135200  records Current Record Size: 4435
Processing: DE_covid_people.pkl.zip
Loaded:  253320  records Current Record Size: 4435
Processing: FL_covid_people.pkl.zip
Loaded:  3789392  records Current Record Size: 4510
Processing: GA_covid_people.pkl.zip
Loaded:  2083181  records Current Record Size: 5859
Processing: GU_covid_people.pkl.zip
Loade

In [115]:
#//*** Aggregate Combined Data
reload(process_covid)

#//*** Run as Needed takes 4minutes to Process
if False:
    #//*** Load County Level Daily Cases and Deaths
    combined_df = process_covid.load_data(file=county_daily_df_filename )
    
    #//*** Aggregate Each County into Monthly Data, save results to a File
    process_covid.create_monthly_data(combined_df,export="z_county_monthly_df.csv.zip")


In [83]:
#//*** Load and combine Summed Outcomes and Monthly Data: Obsolete Since the Monthly Data is inaccurate for Hospitalization numbers
reload(process_covid)
#//*** Load Monthly Summed outcomes - Built from Individual Ethno - demographics values
outcomes_df = process_covid.load_data(filename="summed_outcomes.pkl.zip")

#//*** Load Cases and Deaths Aggregated to Monthly data
monthly_df = process_covid.load_data(filename="z_county_monthly_df.csv.zip",trim_first_date=True,trim_last_date=True)


#//*** Rename Case_month to Date
outcomes_df.columns = ["Date" if item == "case_month" else item for item in list(outcomes_df.columns)]
outcomes_df = outcomes_df[['Date','county_fips_code','case_count','hosp','icu','death','uc']]

#//*** Merge Monthly Cases with Monthly Ethno-Demographic Hospital Data
df = process_covid.merge_df(outcomes_df,monthly_df,by="Date",left_col="county_fips_code", right_col="FIPS")
del df['county_fips_code']
df



print(df['New_Confirm'].sum(),df['case_count'].sum(),(df['case_count'].sum()/ df['New_Confirm'].sum()))

cor_cols = ['hosp','icu','total_vaccinated_percent','total_vaccinated_count','tot_confirm','tot_deaths']
disp_cols = ['Date', 'Combined_Key','Population', 'total_vaccinated_percent', 'total_vaccinated_count', 'tot_confirm', 'tot_deaths', 'New_Confirm', 'hosp','icu','New_Deaths','death', 'case_count']  
         
#'Recip_County', 'FIPS', 'Recip_State' New_Confirm_100k','New_Confirm_avg_daily_100k', 'New_Deaths_100k','New_Deaths_avg_daily_100k'
ca_df = df [df['Recip_State'] == "CA"]
#ca_df = df [df['FIPS'] == 6037]
#tdf = ca_df[ca_df['Date'] == ca_df['Date'].max()].sort_values('New_Deaths_avg_daily_100k',ascending=False)[cor_cols]
#print(ca_df.columns)

#//*** Print Everything to watch it not work
for group in ca_df.groupby("FIPS"):
    display(group[1][disp_cols])
    print(group[1]['New_Confirm'].sum(),group[1]['case_count'].sum())

Elapsed Time: 0s
53652565.0 47178626 0.879335890092114


,Date,Combined_Key,Population,total_vaccinated_percent,total_vaccinated_count,tot_confirm,tot_deaths,New_Confirm,hosp,icu,New_Deaths,death,case_count
92,2021-01-01,"Alameda, California, US",1671329,1.6,26113.0,75589,857,22040.0,606,88,297.0,215,19636
92,2021-02-01,"Alameda, California, US",1671329,6.7,112261.0,82473,1105,6884.0,251,41,251.0,0,5687
92,2021-03-01,"Alameda, California, US",1671329,21.1,351983.0,85291,1215,2818.0,168,24,126.0,0,2972
92,2021-04-01,"Alameda, California, US",1671329,37.1,620327.0,88519,1268,3228.0,143,19,56.0,0,2825
92,2021-05-01,"Alameda, California, US",1671329,54.7,913490.0,90431,1253,1914.0,76,6,48.0,0,1590
90,2021-06-01,"Alameda, California, US",1671329,61.8,1033606.0,91609,1256,1294.0,99,9,39.0,0,1590
92,2021-07-01,"Alameda, California, US",1671329,64.6,1079631.0,97526,1265,5917.0,267,33,16.0,0,8562
92,2021-08-01,"Alameda, California, US",1671329,67.1,1122148.0,111394,1297,13868.0,273,18,32.0,0,12467
92,2021-09-01,"Alameda, California, US",1671329,69.4,1160240.0,117874,1343,6480.0,162,17,46.0,0,5866
92,2021-10-01,"Alameda, California, US",1671329,71.2,1190559.0,121888,1419,4014.0,109,12,92.0,0,3714


216532.0 210612


,Date,Combined_Key,Population,total_vaccinated_percent,total_vaccinated_count,tot_confirm,tot_deaths,New_Confirm,hosp,icu,New_Deaths,death,case_count
93,2021-01-01,"Amador, California, US",39752,1.3,532.0,3237,32,724.0,20,6,8.0,0,585
93,2021-02-01,"Amador, California, US",39752,6.1,2431.0,3496,42,259.0,9,3,11.0,0,195
93,2021-03-01,"Amador, California, US",39752,16.8,6668.0,3588,45,92.0,8,2,3.0,0,105
93,2021-04-01,"Amador, California, US",39752,26.8,10640.0,3683,46,95.0,0,0,1.0,0,59
93,2021-05-01,"Amador, California, US",39752,34.8,13836.0,3720,46,38.0,4,0,0.0,0,42
91,2021-06-01,"Amador, California, US",39752,37.7,14972.0,3789,46,69.0,4,2,0.0,0,71
93,2021-07-01,"Amador, California, US",39752,39.8,15817.0,3911,48,122.0,23,6,2.0,0,224
93,2021-08-01,"Amador, California, US",39752,41.9,16657.0,4598,53,687.0,38,7,5.0,0,667
93,2021-09-01,"Amador, California, US",39752,46.9,18645.0,5157,61,559.0,26,8,8.0,0,571
93,2021-10-01,"Amador, California, US",39752,49.3,19596.0,5579,64,422.0,17,3,3.0,0,373


6630.0 6371


...

140066.0 134268


,Date,Combined_Key,Population,total_vaccinated_percent,total_vaccinated_count,tot_confirm,tot_deaths,New_Confirm,hosp,icu,New_Deaths,death,case_count
140,2021-01-01,"Yolo, California, US",220500,1.5,3375.0,11943,151,3577.0,88,8,37.0,11,3105
140,2021-02-01,"Yolo, California, US",220500,7.2,15976.0,13073,188,1130.0,39,4,39.0,0,806
140,2021-03-01,"Yolo, California, US",220500,18.8,41508.0,13608,197,535.0,27,3,13.0,0,466
140,2021-04-01,"Yolo, California, US",220500,32.9,72501.0,14042,208,468.0,31,7,11.0,0,469
140,2021-05-01,"Yolo, California, US",220500,44.9,99026.0,14390,210,348.0,24,0,5.0,0,283
138,2021-06-01,"Yolo, California, US",220500,50.7,111795.0,14539,213,182.0,11,1,3.0,0,213
140,2021-07-01,"Yolo, California, US",220500,54.4,120009.0,15374,215,835.0,69,10,2.0,0,1228
140,2021-08-01,"Yolo, California, US",220500,56.7,125030.0,18045,234,2671.0,78,6,19.0,0,2582
140,2021-09-01,"Yolo, California, US",220500,59.0,130023.0,19634,242,1589.0,38,6,8.0,0,1447
140,2021-10-01,"Yolo, California, US",220500,60.8,133956.0,20793,250,1159.0,43,12,8.0,0,1126


31256.0 30129


,Date,Combined_Key,Population,total_vaccinated_percent,total_vaccinated_count,tot_confirm,tot_deaths,New_Confirm,hosp,icu,New_Deaths,death,case_count
141,2021-01-01,"Yuba, California, US",78668,0.6,496.0,5146,33,1307.0,69,13,16.0,0,1112
141,2021-02-01,"Yuba, California, US",78668,4.0,3148.0,5550,41,404.0,37,2,8.0,0,343
141,2021-03-01,"Yuba, California, US",78668,10.3,8110.0,5799,44,249.0,19,1,3.0,0,241
141,2021-04-01,"Yuba, California, US",78668,19.3,15155.0,5991,44,192.0,19,4,1.0,0,169
141,2021-05-01,"Yuba, California, US",78668,26.0,20428.0,6190,48,199.0,13,1,5.0,0,191
139,2021-06-01,"Yuba, California, US",78668,31.2,24546.0,6324,51,153.0,19,2,3.0,0,152
141,2021-07-01,"Yuba, California, US",78668,34.0,26729.0,6766,50,442.0,44,8,1.0,0,619
141,2021-08-01,"Yuba, California, US",78668,37.3,29330.0,8436,54,1670.0,81,24,7.0,0,1674
141,2021-09-01,"Yuba, California, US",78668,41.6,32695.0,9595,66,1159.0,76,18,13.0,0,1052
141,2021-10-01,"Yuba, California, US",78668,44.5,35000.0,10373,82,778.0,46,9,16.0,0,710


13230.0 12796
